# Problem set 3

Peijin Li

#### 1.Construct a feature matrix and target array to predict whether an individual defaults on their debt. Convert the categorical variables to binary numeric values.

In [ ]:
# Data Management/Investigation
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
### Read in data
default = pd.read_csv("default.csv")
##Construct a feature matrix 
X_98 = default.drop('default',axis=1)
##target array to predict whether an individual defaults on their debt.
y_98 = default['default']
#print out the feature matrix 
X_98.head()

In [ ]:
##Convert the categorical variables to binary numeric values
from sklearn.preprocessing import LabelEncoder
# Instantiate class
le = LabelEncoder()
le_stu = LabelEncoder()
# Apply to purchased column
default["default_dum"] = le.fit_transform(default["default"])
default["student_dum"] = le_stu.fit_transform(default["student"])
df=default.filter(["default_dum","student_dum","balance","income"])
df=df.rename(columns={"default_dum":"default","student_dum":"student"})
df.head()

#### 2.Calculate the base rate for the positive class, default=1. In other words, what proportion of borrowers in the dataset are classified as having defaulted? What challenges might this present for your analysis?

In [5]:
# Calculate the base rate for the positive class, default=1.
positive=df['default'].value_counts()[1]
negative=df['default'].value_counts()[0]
positive_class_base_rate=positive/(positive+negative)
positive_class_base_rate

0.0333

This leads to the class imbalance problem. Machine learning algorithms assume data is equally distributed. So, when we have a class imbalance, the machine learning classifier tends to be more biased towards the majority class, causing bad classification of the minority class.It occurs because the conventional machine learning algorithm cost function constantly attempts to optimize quantities such as error rate without considering the data distribution.

#### 3.Split the data into training and test sets, holding out 20% of the data.

In [5]:
# Train-Test split (just using Pandas)
#Create a training and test set with 20% of the data held out for testing.
train = df.sample(frac=.80).reset_index(drop=True)
test = df.drop(train.index).reset_index(drop=True)

#### 4.Fit each of the following models using the training data, and compute the accuracy rate using the test data.

(a) Logistic Regression

In [6]:
# Fit Logistic Regression models using the training data
# Set feature (now use all feautures) and target
X = df.drop("default", axis=1)
y = df[["default"]]
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98)
# Import class from Logistic_model module within sklearn
from sklearn.linear_model import LogisticRegression
# Instantiate model object
lr = LogisticRegression()
# Fit model to training data
lr.fit(X_train, y_train)
# Make predictions on unseen data using the test set
y_test_pred = lr.predict(X_test)
##remove the extra braces
import numpy as np
y_test_pred=np.ravel(y_test_pred)

In [37]:
#compute the accuracy rate using the test data
# Import metrics from sklearn
from sklearn import metrics
print("Accuracy score of Logistic Regression model:", metrics.accuracy_score(y_test,y_pred=y_test_pred))

Accuracy score of Logistic Regression model: 0.9665


(b) Support Vector Machine (set C = 1 and probability=True)

In [9]:
# Import class from LinearSVC_model module within sklearn
from sklearn.svm import SVC
# Instantiate model object
svm=SVC(C=1, probability = True)
# Fit model to training data
svm.fit(X_train, y_train)
# Make predictions on unseen data using the test set
y_test_pred_1=svm.predict(X_test)

In [36]:
##compute the accuracy rate using the test data
print("Accuracy score of Support Vector Machine model:", metrics.accuracy_score(y_test,y_pred=y_test_pred_1))

Accuracy score of Support Vector Machine model: 0.9675


#### 5.For each of the models in #4, generate predictions and compute the recall score. Discuss the performance of the models with respect to accuracy and recall.

In [11]:
#generate predictions with Logistic Regression model 
y_test_pred = lr.predict(X_test)
#generate predictions with Support Vector Machine model 
y_test_pred_1=svm.predict(X_test)

In [35]:
##compute the recall score of Logistic Regression
print("Recall score of Logistic Regression model:", metrics.recall_score(y_test,y_pred=y_test_pred))
##compute the recall score of Support Vector Machine
print("Recall score of Support Vector Machine model:", metrics.recall_score(y_test,y_pred=y_test_pred_1))

Recall score of Logistic Regression model: 0.18461538461538463
Recall score of Support Vector Machine model: 0.0


Discuss the performance of the models:Accuracy=(TP+TN)/(TP+FP+TN+FN). The accuracy score of Logistic Regression model is 0.9665, and the accuracy score of Support Vector Machine model is 0.9675,slightly higher than the Logistic Regression model. This means compared with Logistic Regression model, Support Vector Machine model's prediction is more accurate. REC(recall score) =TP/(FN+TP). The larger of REC, the greater(so, in this case,Support Vector Machine model performs worse), which means it does worse on minimizing the chances of missing positive cases. Actually, Support Vector Machine model predicates no case to be positive.

#### 6.Compute and show a confusion matrix. Is either recall or precision more important than the other in this domain? Why or why not? 

In [15]:
#Confusion Matrix of Logistic Regression model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred)
# Assess using relevant metrics: confusion matrix   true/pred   #  no/no |  no/yes # yes/no |  yes/yes
pd.DataFrame(confusion_matrix(y_test, y_test_pred),
            columns=["Predicted negative", "Predicted positive"],
            index=["Actual negative","Actual positive"]).style.background_gradient(cmap="summer")

,Predicted negative,Predicted positive
Actual negative,1921,14
Actual positive,53,12


In [16]:
#Confusion Matrix of Support Vector Machine model
confusion_matrix(y_test, y_test_pred_1)
# Assess using relevant metrics: confusion matrix   true/pred   #  no/no |  no/yes # yes/no |  yes/yes
pd.DataFrame(confusion_matrix(y_test, y_test_pred_1),
            columns=["Predicted negative", "Predicted positive"],
            index=["Actual negative","Actual positive"]).style.background_gradient(cmap="summer")

,Predicted negative,Predicted positive
Actual negative,1935,0
Actual positive,65,0


In this domain, recall is more important. Because it actually is more costly than predicting something is not default and it turned out to be. REC(recall score) =TP/(FN+TP) can be used to minimize the chances of missing positive cases.

#### 7.For this part of the assignment, you will use data that oversamples the positive class (subset data so default=1 ).

(a) Create a new dataframe that retains all negative class observations and oversamples the positive class with replacement so that the base rate for the positive class is 50%. This dataframe should have 19,334 observations (n+ = 9, 667, n− =9, 667).

In [17]:
#Create a new dataframe that retains all negative class observations 
df_negative=df.loc[df.default == 0]
df_negative.shape

(9667, 4)

In [18]:
#oversamples the positive class with replacement so that the base rate for the positive class is 50%.
# Import class
from imblearn.over_sampling import RandomOverSampler
# Instantiate
over = RandomOverSampler(random_state=98)
# split the data
X_1 = df.drop("default", axis=1)
y_1 = df[["default"]]
# Fit features and target
x_over, y_over = over.fit_resample(X_1, y_1)
# Check target classes
print(f"Oversampled:\n{y_over.value_counts()}\mZ")

Oversampled:
default
0          9667
1          9667
dtype: int64\mZ


(b) Repeat #3, #4 and #5 with this new data—fit each of the models on the training data and compute their accuracy and recall scores on the test data.

In [19]:
# combine x_over and y_over
df_oversample=pd.concat([x_over,y_over],axis=1)

In [20]:
##(3)
# Train-Test split (just using Pandas)
#Create a training and test set with 20% of the data held out for testing.
train_1 = df_oversample.sample(frac=.80).reset_index(drop=True)
test_1 = df_oversample.drop(train_1.index).reset_index(drop=True)
##(4.a)
# Fit Logistic Regression models using the training data
# Set feature (now use all feautures) and target
X_os = df_oversample.drop("default", axis=1)
y_os = df_oversample[["default"]]
# Split data
X_train_os, X_test_os, y_train_os, y_test_os = train_test_split(X_os, y_os, test_size=0.2, random_state=98)
# Instantiate model object
lr_os = LogisticRegression()
# Fit model to training data
lr_os.fit(X_train_os, y_train_os)
# Make predictions on unseen data using the test set
y_test_pred_os = lr_os.predict(X_test_os)
##remove the extra braces
y_test_pred_os=np.ravel(y_test_pred_os)

In [21]:
##(4.b)
# Instantiate model object
svm_os=SVC(C=1, probability = True)
# Fit model to training data
svm_os.fit(X_train_os, y_train_os)
# Make predictions on unseen data using the test set
y_test_pred_1_os=svm_os.predict(X_test_os)

In [34]:
##(5)
#compute the accuracy rate using the test data
print("Accuracy score of Logistic Regression model:", metrics.accuracy_score(y_test_os,y_pred=y_test_pred_os))
##compute the recall score of Logistic Regression
print("Recall score of Logistic Regression model:", metrics.recall_score(y_test_os,y_pred=y_test_pred_os))

Accuracy score of Logistic Regression model: 0.8709594000517197
Recall score of Logistic Regression model: 0.8934725848563969


In [33]:
##compute the accuracy rate using the test data
print("Accuracy score of Support Vector Machine model:", metrics.accuracy_score(y_test_os,y_pred=y_test_pred_1_os))
##compute the recall score of Support Vector Machine
print("Recall score of Support Vector Machine model:", metrics.recall_score(y_test_os,y_pred=y_test_pred_1_os))

Accuracy score of Support Vector Machine model: 0.8675976208947505
Recall score of Support Vector Machine model: 0.9044386422976501


(c) Discuss the performance tradeoff associated with oversampling.

For both models, with oversampling, the accuracy score of the prediction decreses, and the recall score increases. The random oversampling may increase the likelihood of overfitting occurring, since it makes exact copies of the minority class examples. In this way, a symbolic classifier, for instance, might construct rules that are apparently accurate, but actually cove one replicated example. So, when we apply thie model to fit the testing data, the accuracy decreases. With more minority class available, the model performs better on identify positive cases, which explains the increases of recall score.

#### 8.For this part of the assignment, you will use data that undersamples the negative class (subset data so default=0 ).

(a) Create a new dataframe that retains all positive class observations and undersamples the negative class so that the base rate for the positive class is 50%. This dataframe should have 666 observations (n+ = 333, n− = 333).

In [24]:
##Create a new dataframe that retains all positive class observations
df_positive=df.loc[df.default == 1]
df_positive.shape

(333, 4)

In [25]:
#undersamples the negative class with replacement so that the base rate for the positive class is 50%.
# Import class
from imblearn.under_sampling import RandomUnderSampler
# Instantiate
under = RandomUnderSampler(random_state=98)
# split the data
X_2 = df.drop("default", axis=1)
y_2 = df[["default"]]
# Fit features and target
x_under, y_under = under.fit_resample(X_2, y_2)
# Check target classes
print(f"Undersampled:\n{y_under.value_counts()}\mZ")

Undersampled:
default
0          333
1          333
dtype: int64\mZ


(b) Repeat #3, #4 and #5 with this new data—fit each of the models on the training data and compute their accuracy and recall scores on the test data.

In [26]:
# combine x_under and y_under
df_undersample=pd.concat([x_under,y_under],axis=1)

In [27]:
##(3)
# Train-Test split (just using Pandas)
#Create a training and test set with 20% of the data held out for testing.
train_2 = df_undersample.sample(frac=.80).reset_index(drop=True)
test_2 = df_undersample.drop(train_2.index).reset_index(drop=True)
##(4.a)
# Fit Logistic Regression models using the training data
# Set feature (now use all feautures) and target
X_ud = df_undersample.drop("default", axis=1)
y_ud = df_undersample[["default"]]
# Split data
X_train_ud, X_test_ud, y_train_ud, y_test_ud = train_test_split(X_ud, y_ud, test_size=0.2, random_state=98)
# Instantiate model object
lr_ud = LogisticRegression()
# Fit model to training data
lr_ud.fit(X_train_ud, y_train_ud)
# Make predictions on unseen data using the test set
y_test_pred_ud = lr_os.predict(X_test_ud)
##remove the extra braces
y_test_pred_ud=np.ravel(y_test_pred_ud)

In [28]:
##(4.b)
# Instantiate model object
svm_ud=SVC(C=1, probability = True)
# Fit model to training data
svm_ud.fit(X_train_ud, y_train_ud)
# Make predictions on unseen data using the test set
y_test_pred_1_ud=svm_os.predict(X_test_ud)

In [32]:
##(5)
#compute the accuracy rate using the test data
print("Accuracy score of Logistic Regression model:", metrics.accuracy_score(y_test_ud,y_pred=y_test_pred_ud))
##compute the recall score of Logistic Regression
print("Recall score of Logistic Regression model:", metrics.recall_score(y_test_ud,y_pred=y_test_pred_ud))

Accuracy score of Logistic Regression model: 0.8432835820895522
Recall score of Logistic Regression model: 0.8382352941176471


In [31]:
##compute the accuracy rate using the test data
print("Accuracy score of Support Vector Machine model:", metrics.accuracy_score(y_test_ud,y_pred=y_test_pred_1_ud))
##compute the recall score of Support Vector Machine
print("Recall score of Support Vector Machine model:", metrics.recall_score(y_test_ud,y_pred=y_test_pred_1_ud))

Accuracy score of Support Vector Machine model: 0.8283582089552238
Recall score of Support Vector Machine model: 0.8529411764705882


(c) Discuss the performance tradeoff associated with undersampling.

For both models, with undersampling, the accuracy score of the prediction decreses, and the recall score increases. In random undersampling, vast quantities of data are discarded.This can be highly problematic, as the loss of such data can make the decision boundary between the minority and majority instances harder to learn, resulting in a loss in classification performance. So, when we apply the model to fit the testing data, the accuracy decreases. With a more balanced dataset, the model performs better on identify positive cases, which explains the increases of recall score. 

#### 9.You have now examined the following scenarios:
Original data (n+ = 333, n− = 9, 667)
Oversampling the positive class (n+ = 9, 667, n− = 9, 667)
Undersampling the negative class (n+ = 333, n− = 333)
Based on your analysis, which strategy would you employ to address the imbalanced class problem? Which model would you select as the “best” model? Explain your rationale.

In this case, I woule like to select oversampling the positive class model as the “best” model. First of all, I would like to discard the model of the original data, because with the inbalanced class problem, the model performs poor with a recall score of 0(Support Vector Machine model) and 0.184(Logistic Regression model), which means in this domain, there is a high chance that the model miss positive cases in its prediction. I think oversampling is better than undersampling, because the accuracy score of oversampling is higher than undersampling. This means oversampling performs better than undersampling on predicting unseen data. This is because undersampling discards vast quantities of data, render the decision boundary between the minority and majority instances harder to learn, resulting in a loss in classification performance. And with more data to learn, oversampling model also performs better than undersampling model referring to recall scores, which means oversampling model performs better on identify positive cases.